# Laptop specifications scrapper
Scrap comparez-malin.com for laptop specification

In [ ]:
import os
import time
import json
from urllib.request import urlopen
from urllib.parse import urljoin
from urllib.error import HTTPError

from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

## Utils function

In [ ]:
def save_and_reload_df(func):
    """
    Decorator that saves the dataframe computed by the function
    and loads it if it was already saved
    """
    def func_wrapper(*args, overwrite=False, **kwargs):
        csv_path = "data/{}.csv".format(func.__name__)
        if not os.path.exists(csv_path) or overwrite:
            df = func(*args, **kwargs)
            df.to_csv(csv_path)
        else:
            print("Reading dataframe from {}".format(csv_path))
            df = pd.read_csv(csv_path, index_col=0)
        return df
    return func_wrapper


def get_specs(url):
    """Return specs as a dictionary"""
    html_doc = urlopen(url)
    html_doc = html_doc.read()
    soup = BeautifulSoup(html_doc, "html.parser")
    soup = soup.find("div", {"id": "specs"})
    specs = {}
    for spec in soup.find_all("tr"):
        key, value = extract_spec(spec)
        if key:
            specs[key] = value
    
    return specs


def extract_spec(spec):
    key = spec.find("th", {"scope": "row"})
    if key:
        key = key.text
        key = key.replace("\n", " ").strip()
        value = spec.find("td").text
        value = value.replace("\n", " ").strip()
        value = value.replace(u'\xa0', u' ')
    else:
        return None, None
    return key, value


def get_laptop_urls_in_page(page_url):
    root_url = "http://www.comparez-malin.fr/informatique/pc-portable/"
    html_doc = urlopen(page_url).read()
    soup = BeautifulSoup(html_doc, "html.parser")
    laptop_blocks = soup.find_all("div", {"class": "product"})
    specs_urls = {}
    for block in laptop_blocks:
        try:
            key = block["id"]
            url = block.find("a", {"class": "white"})["href"]
            if "tablette" not in url:
                url = urljoin(root_url, url.split('/')[-1])
                specs_urls[key] = url
        except KeyError:
            pass
    return specs_urls


def add_columns(df, columns):
    """Add columns to a dataframe"""
    # Remove columns that are already there
    columns = set(columns) - set(df.columns)
    df_columns = pd.DataFrame(columns=columns)
    df = df.join(df_columns, how='outer')
    return df

@save_and_reload_df
def get_laptops_urls():
    """Get links to each laptop page in a dataframe"""
    root_url = "http://www.comparez-malin.fr/informatique/pc-portable/{}"
    n = 265
    specs_urls = {}
    for i in tqdm(range(n)):
        page_url = root_url.format(i+1)
        specs_urls.update(get_laptop_urls_in_page(page_url))

    # Convert urls to dataframe
    s = pd.Series(specs_urls, name='url')
    df = s.to_frame()
    df.to_csv(csv_path)
    return df

@save_and_reload_df
def get_all_laptops_specs(df_laptops_urls, overwrite=False):
    """Get specs for all laptops urls"""
    df = df_laptops_urls
    # Initialize columns
    url = df.iloc[0]["url"]
    specs = get_specs(url)
    columns = set(specs.keys())
    df = add_columns(df, columns)
    columns = set(df.columns)

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if row.isnull().values[1:].all():
            url = row["url"]
            specs = get_specs(url)
            if len(specs) == 0:
                print(url)
                pass
            specs["url"] = url
            new_cols = set(specs.keys())
            if (new_cols != columns):
                df = add_columns(df, new_cols - columns)
                columns = set(df.columns)
            df.loc[index] = specs
    df.to_csv('data/all_specs.csv')
    return df

def get_cpu_benchmark(cpu_name):
    root_url = "http://www.cpubenchmark.net/cpu.php?cpu={}"
    try:
        url = root_url.format(cpu_name.replace(" ", "+"))
        html_doc = urlopen(url).read()
        soup = BeautifulSoup(html_doc, "html.parser")
        # Square with perf and single thread rating
        soup = soup.find("td", {"style": "text-align: center"})
        benchmark = int(soup.find("span").text)
    except HTTPError:
        benchmark = None
    return benchmark

@save_and_reload_df
def get_cpu_dataframe():
    cpus = df["processeur"].unique()
    df_cpu = pd.DataFrame(cpus, columns=["processeur"])
    for index, row in tqdm(df_cpu.iterrows(), total=df_cpu.shape[0]):
        benchmark = get_cpu_benchmark(row["processeur"])
        df_cpu.loc[index, "cpu_benchmark"] = benchmark
    return df_cpu

In [ ]:
def test_get_specs():
    url = "http://www.comparez-malin.fr/informatique/pc-portable/asus-zenbook-3-ux390ua-gs039r.html"
    specs = get_specs(url)
    retrieved_specs_keys = list(specs.keys())
    filename = "data/test_get_specs_keys.txt"
    with open(filename, "r") as fp:
        true_specs_keys = fp.readlines()
    true_specs_keys = [key.replace("\n", " ").strip() for key in true_specs_keys]
    not_retrieved = set(true_specs_keys) - set(retrieved_specs_keys)
    assert len(not_retrieved) == 0, "Not retrieved: {}".format(not_retrieved)
    not_asked = set(retrieved_specs_keys) - set(true_specs_keys)
    assert len(not_asked) == 0, "Not asked: {}".format(not_asked)
test_get_specs()

Get links to each laptop page in a  dataframe

In [ ]:
df_laptops_urls = get_laptops_urls()

Get specs for all laptops

In [ ]:
df = get_all_laptops_specs(df_laptops_urls)
df.columns = df.columns.str.lower().str.replace(" ", "_").str.replace(".", "")

Get CPU benchmarks

In [ ]:
df_cpu = get_cpu_dataframe()
df = df.merge(df_cpu)

Processing

In [ ]:
def process_prix_public(price):
    price = price.strip("€")
    price = price.replace(" ", "")
    price = price.replace(",", ".")
    price = float(price)
    return price

In [ ]:
col_methods = {
    "prix_public": process_prix_public
}

for col, method in col_methods.items():
    df[col] = df[col].apply(lambda x: method(x))
    
df[["cores", "min_freq", "max_freq"]] = df["fréquence"].str.split(expand=True)[[0,2,4]].astype(float)
df["pdt_max"] = df["pdt_max"].str.split(expand=True)[0].astype(int)

In [ ]:
def process_disque_dur(string):
    string = string.replace("(", "").replace(")", "").replace(u'\xa0', u' ')
    #string = string.replace("\xa0", " ")
    sshd = "cache SSD" in string
    string = string.replace("cache SSD", "")
    dd_size = 0
    dd_speed = 0
    if "tr/min" in string:
        splitted = string.split("tr/min")[0].split()
        dd_size = int(splitted[0])
        dd_speed = int(splitted[-1])
    ssd_size = 0
    print(repr(string))
    if "Go SSD" in string:
        ssd_size = int(string.split("Go SSD")[0].split()[-1])
    return dd_size, dd_speed, sshd, ssd_size

df[["dd_size", "dd_speed", "sshd", "ssd_size"]] = 0
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    print((row["disque_dur"]))
    print(process_disque_dur(row["disque_dur"]))
    dd_size, dd_speed, sshd, ssd_size = process_disque_dur(row["disque_dur"])
    df.loc[index, "dd_size"] = dd_size
    df.loc[index, "dd_speed"] = dd_speed
    df.loc[index, "sshd"] = sshd
    df.loc[index, "ssd_size"] = ssd_size

Scoring

In [ ]:
def get_score(row):
    score = 0
    coefs = {
        "cpu_benchmark": 0.8,
        "prix_public": 0.5,
        "min_freq": 0.2,
        "max_freq": 0.4,
        "pdt_max": 0.25
    }
    score += row["cpu_benchmark"]/4000 * coefs["cpu_benchmark"]
    score += row["min_freq"]/2.5 * coefs["min_freq"]
    score += row["max_freq"]/3.3 * coefs["max_freq"]
    score += row["pdt_max"]/38 * -coefs["pdt_max"]

    score /= (row["prix_public"]/800)**coefs["prix_public"]
    return score

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.loc[index, "score"] = get_score(row)

In [ ]:
pd.set_option('display.max_colwidth', -1)
df["score"] /= df["score"].max()
cols_to_show = ["url", "score", "cpu_benchmark", "prix_public"]
df.sort_values(by="score", ascending=False)[cols_to_show].head()